In [1]:
import numpy as np
import sciopy as sc

In [2]:
sc.available_serial_ports()

['COM1', 'COM3']

In [3]:
ser = sc.connect_COM_port('COM3')

Connection to COM3 is established.


In [4]:
def GeneralSystemMessages(serial, msg_old:list=[]): # Why returns this function none?!?!?!
    """Reads the message buffer of a serial connection. Also prints out the general system message."""
    msg_dict = {
        b"\x02": "Timeout: Communication-timeout (less data than expected)",
        b"\x04": "Wake-Up Message: System boot ready",
        b"\x11": "TCP-Socket: Valid TCP client-socket connection",
        b"\x81": "Not-Acknowledge: Command has not been executed",
        b"\x82": "Not-Acknowledge: Command could not be recognized",
        b"\x83": "Command-Acknowledge: Command has been executed successfully",
        b"\x84": "System-Ready Message: System is operational and ready to receive data",
        b"\x91": "Data holdup: Measurement data could not be sent via the master interface",
    }
    msg_new = []
   
    start_cmd_tag = serial.read() # Read start tag.
    cmd_msg_len = int.from_bytes(serial.read(), "big") # Read length of the message.
    if msg_old: # msg_old has elements:
        #msg_new.extend(start_cmd_tag)
        msg_new.extend([serial.read() for i in range(cmd_msg_len)])
        end_cmd_tag = serial.read()# Read message end tag.
        #msg_new.extend(end_cmd_tag)
        msg_old.extend(msg_new) # Reads the serial message in Hex and adds to msg_old
    else:
        #msg_new.extend(start_cmd_tag)
        msg_new.extend([serial.read() for i in range(cmd_msg_len)]) # Reads the serial message in Hex.  
        end_cmd_tag = serial.read()# Read message end tag.
        #msg_new.extend(end_cmd_tag)
        msg_old = []
        msg_old = msg_new
    #msg_new = [int.from_bytes(m, "big") for m in msg_new] # Convert message to integer.
    #if start_cmd_tag == end_cmd_tag: # Concludes with the comparison of start and end tag   
    #print("start tag:",start_cmd_tag)
    #print("msg_new len:",cmd_msg_len)
    #print("end tag:",end_cmd_tag)

    if end_cmd_tag != b'\x18':
        print("Recall GeneralSystemMessages(serial, msg_new)\n")
        GeneralSystemMessages(serial, msg_new)
    else:
        print(msg_dict[msg_new[0]])
        print("msg_new:",msg_new)
        print("msg_old:",msg_old)
        return msg_old

In [5]:
def GetFirmwareIDs(serial):
    """Get firmware IDs"""
    serial.write(bytearray([0xD2, 0x00, 0xD2]))
    GeneralSystemMessages(serial)

In [6]:
GetFirmwareIDs(ser)

Recall GeneralSystemMessages(serial, msg_new)

Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x01', b'\x11', b'\x00', b'\x19', b'\x00', b'\x0b', b'@', b'\x00', b'\t', b'\x00', b'%', b'\x02', b'\xc9', b'\x00', b'\x0f', b'\x01', b'\x11', b'\x00', b'\x19', b'\x00', b'\x0b', b'@', b'\x00', b'\t', b'\x00', b'%', b'\x02', b'\xc9', b'\x00', b'\x0f', b'\x83']


In [67]:
def SoftwareReset(serial):
    serial.write(bytearray([0xA1, 0x00, 0xA1]))
    GeneralSystemMessages(serial)

In [68]:
SoftwareReset(ser)

Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']


In [69]:
def ResetMeasurementSetup(serial):
    serial.write(bytearray([0xB0, 0x01, 0x01, 0xB0]))
    callback = GeneralSystemMessages(serial)
    print(callback)

In [70]:
ResetMeasurementSetup(ser)

SerialException: WriteFile failed (PermissionError(13, 'Das Gerät erkennt den Befehl nicht.', None, 22))

# Final Running schedule

In [7]:
def Config_01(serial) -> None:
    serial.write(bytearray([0xB0, 0x01, 0x01, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x02, 0x00, 0x01, 0xB0])) # Burst count
    GeneralSystemMessages(ser)
    serial.write(
        bytearray(
            [0xB0, 0x09, 0x05, 0x3F, 0x50, 0x62, 0x4D, 0xD2, 0xF1, 0xA9, 0xFC, 0xB0]
        )
    )
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x02, 0x0D, 0x01, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x09, 0x01, 0x00, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x08, 0x01, 0x01, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x02, 0x0C, 0x01, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x05, 0x03, 0x40, 0x00, 0x00, 0x00, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(
        bytearray(
            [
                0xB0,
                0x0C,
                0x04,
                0x44,
                0x7A,
                0x00,
                0x00,
                0x44,
                0x7A,
                0x00,
                0x00,
                0x00,
                0x01,
                0x00,
                0xB0,
            ]
        )
    )
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x01, 0x02, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x02, 0x03, 0xB0]))
    GeneralSystemMessages(ser) 
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x03, 0x04, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x04, 0x05, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x05, 0x06, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x06, 0x07, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x07, 0x08, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x08, 0x09, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x09, 0x0A, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0A, 0x0B, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0B, 0x0C, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0C, 0x0D, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0D, 0x0E, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0E, 0x0F, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x0F, 0x10, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x10, 0x11, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x11, 0x12, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x12, 0x13, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x13, 0x14, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x14, 0x15, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x15, 0x16, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x16, 0x17, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x17, 0x18, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x18, 0x19, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x19, 0x1A, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1A, 0x1B, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1B, 0x1C, 0xB0]))
    GeneralSystemMessages
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1C, 0x1D, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1D, 0x1E, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1E, 0x1F, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x1F, 0x20, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB0, 0x03, 0x06, 0x20, 0x01, 0xB0]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB1, 0x01, 0x03, 0xB1]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB2, 0x02, 0x01, 0x01, 0xB2]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB2, 0x02, 0x03, 0x01, 0xB2]))
    GeneralSystemMessages(ser)
    serial.write(bytearray([0xB2, 0x02, 0x02, 0x01, 0xB2]))
    GeneralSystemMessages(ser)
    GeneralSystemMessages(ser)
    #serial.write(bytearray([0xB4, 0x01, 0x01, 0xB4]))
    #sleep(10)
    #serial.write(bytearray([0xB4, 0x01, 0x00, 0xB4]))


In [8]:
import time
def StartStopMeasurement(serial):
    serial.write(bytearray([0xB4, 0x01, 0x01, 0xB4]))
    callback = GeneralSystemMessages(serial)
    print(callback)
    time.sleep(5)
    serial.write(bytearray([0xB4, 0x01, 0x00, 0xB4]))
    GeneralSystemMessages(serial)
    
def StopMeasurement(serial):
    serial.write(bytearray([0xB4, 0x01, 0x00, 0xB4]))
    GeneralSystemMessages(serial)

In [9]:
Config_01(ser)

Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
Command-Acknowledge:

In [10]:
#GeneralSystemMessages(ser)

In [11]:
#ser.read()

In [12]:
StartStopMeasurement(ser)

Command-Acknowledge: Command has been executed successfully
msg_new: [b'\x83']
msg_old: [b'\x83']
[b'\x83']
Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)

Recall GeneralSystemMessages(serial, msg_new)


KeyError: b'\x92'

In [18]:
StopMeasurement(ser)

start tag: b'\x02'
msg_new len: 3
None
None


In [41]:
ser.read()

b'\xc0'

In [ ]:
mx = 0
while ser.read() != None: #4000+
    ser.read()
    mx += 1
print(mx)

In [6]:
int.from_bytes(b'0x01', byteorder='little')

825260080

In [11]:
int.from_bytes(bytes(0xA2), byteorder='little')

0

In [13]:
int(0x18)

24